# Projeto 1 - Ciência dos Dados
# Classificador Xbox

Nome: Alex Steijntjes

Nome: Alexandre Gonçalves Cury

___
Carregando algumas bibliotecas:

In [39]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [40]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
/Users/alexandrecury/Documents/GitHub/p1_cdados2021


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [41]:
filename = 'Xbox Series.xlsx'

In [42]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Classificado
0,@xboxbr me dá um xbox séries s 👉👈😎,1.0
1,@dhominy00274063 @gabriel99198382 mas o series...,1.0
2,xbox series s\n\nhttps://t.co/ujqwy55mkr\n\n( ...,0.0
3,marvel’s avengers | também será lançado no #ps...,0.0
4,primeiros testes alpha do dolby vision no xbox...,0.0


In [43]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste
0,um patch de manutenção está sendo implantado n...
1,microsoft convida anthony mackie para uma roda...
2,@sk8jinx antes eu queria o xbox series x\n\nma...
3,passa la no meu video familia\n\n tipo gringa💃...
4,quer ganhar um ‘refrigerador’ do xbox series x...


___
## Classificador automático de sentimento


No projeto 1 de ciência de dados, decidimos analizar se os Tweets de usuários do Xbox indicam uma satisfação ou não com o produto

___
### Montando um Classificador Naive-Bayes

In [44]:
#Limpeza
def clean(tweet):
    return ''.join([i if ord(i) < 128 else '' for i in tweet])
train["Treinamento"] = train["Treinamento"].apply(clean)
train["Treinamento"] = train["Treinamento"].apply(lambda x: x.lower())
train["Treinamento"] = train["Treinamento"].apply(lambda x: x.replace('.',' '))
train["Treinamento"] = train["Treinamento"].apply(lambda x: x.replace('\n',' '))
train["Treinamento"] = train["Treinamento"].apply(lambda x: x.replace('?',' '))
train["Treinamento"] = train["Treinamento"].apply(lambda x: x.replace('!',' '))
train["Treinamento"] = train["Treinamento"].apply(lambda x: x.replace('"',' '))
train["Treinamento"] = train["Treinamento"].apply(lambda x: x.replace(';',' '))
train["Treinamento"] = train["Treinamento"].apply(lambda x: x.replace('#',' '))
train["Treinamento"] = train["Treinamento"].apply(lambda x: x.replace('&amp',' '))
train["Treinamento"] = train["Treinamento"].apply(lambda x: x.replace(',',' '))
train.Classificado.value_counts(True)

0.0    0.64257
1.0    0.35743
Name: Classificado, dtype: float64

In [45]:
#Definir 0 como irrelevante e 1 como relevante, variavel categorica

train["Classificado"] = train["Classificado"].astype('category')
train["Classificado"].cat.categories = ['relevante', 'irrelevante']

In [46]:
#Relevante e Irrelevante
relevante = train["Classificado"] == 'relevante'
data1 = train.loc[relevante, :]
total_relevante = data1.Classificado.value_counts()[0]

print("O numero de dados relevantes é: {0}".format(total_relevante))

irrelevante = train["Classificado"] == 'irrelevante'
data2 = train.loc[irrelevante, :]
total_irrelevante = data2.Classificado.value_counts()[1]

print("O numero de dados irrelevantes antes de filtro é: {0}".format(total_irrelevante))

O numero de dados relevantes é: 320
O numero de dados irrelevantes antes de filtro é: 178


In [47]:
#Toal de Palavras
total_palavras = total_relevante+total_irrelevante
total_palavras

498

In [48]:
#Tabela Relevante
relevante = train["Classificado"] == 'relevante' # pegar somente dados classificados como relevante
data_relevante = train.loc[relevante, :] # filtrar no dataframe
tweet_relevante = pd.Series(data_relevante.Treinamento) #gera uma series de dados relevantes
tweet_relevante = pd.DataFrame(tweet_relevante) #transforma a series em um dataframe
tweet_relevante["Treinamento"] = tweet_relevante["Treinamento"].apply(clean)
palavras_relevantes = tweet_relevante.Treinamento.str.cat() # transforma o dataframe em uma lista de palavras
palavras_relevantes = palavras_relevantes.split()
palavras_relevantes = pd.Series(palavras_relevantes)


abs_relevante = palavras_relevantes.value_counts()
total_relevante = abs_relevante.sum()
print(abs_relevante)
print("Numero total de palavras relevantes: ", total_relevante)

xbox         350
https://t    312
series       289
o            183
e            179
            ... 
terceiro       1
gog            1
cima           1
atualiza       1
what           1
Length: 1617, dtype: int64
Numero total de palavras relevantes:  7657


In [49]:
print('Tabela Relativa')

rel_relevante = palavras_relevantes.value_counts(True)
print(rel_relevante)
print(rel_relevante.sum())

Tabela Relativa
xbox         0.045710
https://t    0.040747
series       0.037743
o            0.023900
e            0.023377
               ...   
terceiro     0.000131
gog          0.000131
cima         0.000131
atualiza     0.000131
what         0.000131
Length: 1617, dtype: float64
1.0


In [50]:
#Tabela Irrelevante

irrelevante = train["Classificado"] == 'irrelevante' # pegar somente dados classificados como relevante
data_irrelevante = train.loc[irrelevante, :] # filtrar no dataframe
tweet_irrelevante = pd.Series(data_irrelevante.Treinamento) #gera uma series de dados relevantes
tweet_irrelevante = pd.DataFrame(tweet_irrelevante) #transforma a series em um dataframe
tweet_irrelevante["Treinamento"] = tweet_irrelevante["Treinamento"].apply(clean)
palavras_irrelevantes = tweet_irrelevante.Treinamento.str.cat() # transforma o dataframe em uma lista de palavras
palavras_irrelevantes = palavras_irrelevantes.split()
palavras_irrelevantes = pd.Series(palavras_irrelevantes)


abs_irrelevante = palavras_irrelevantes.value_counts()
total_irrelevante = abs_irrelevante.sum()
print(abs_irrelevante)
print("Numero total de palavras irrelevantes: ", total_irrelevante)

xbox           216
o              181
de             179
series         170
no             137
              ... 
postou           1
xboxseriesx      1
empurrar         1
enorme           1
podemos          1
Length: 1173, dtype: int64
Numero total de palavras irrelevantes:  5447


In [51]:
#Tabela Relativa
rel_irrelevante = palavras_irrelevantes.value_counts(True)
print(rel_irrelevante)
print(rel_irrelevante.sum())

xbox           0.039655
o              0.033229
de             0.032862
series         0.031210
no             0.025151
                 ...   
postou         0.000184
xboxseriesx    0.000184
empurrar       0.000184
enorme         0.000184
podemos        0.000184
Length: 1173, dtype: float64
0.9999999999999999


In [52]:
#Resultado das palavras
print("Numero total de palavras relevantes é: ", total_relevante)
print(55*"-")
print("Numero total de palavras irrelevantes é: ", total_irrelevante)
print(55*"-")
total2 = total_relevante+total_irrelevante # apos o filtro

print("Numero total de palavras apos o filtro/ajustes é: ",  total2)

Numero total de palavras relevantes é:  7657
-------------------------------------------------------
Numero total de palavras irrelevantes é:  5447
-------------------------------------------------------
Numero total de palavras apos o filtro/ajustes é:  13104


In [53]:
#Probabilidade de ser relevante ou irrelevante

prob_rel = total_relevante/total2
prob_irr = total_irrelevante/total2
checagem = prob_rel+prob_irr
print("A probabilidade de ser relevante é: ", prob_rel) # palavras
print("A probabilidade de ser irrelevante é: ", prob_irr) # palavras
print("Checagem: ",checagem)


A probabilidade de ser relevante é:  0.5843253968253969
A probabilidade de ser irrelevante é:  0.4156746031746032
Checagem:  1.0


___
### Classificador Naive-Bayes

In [54]:
prob_tweet_rel = 1
prob_tweet_irr = 1



lista_rel = []
lista_irr = []

a = "relevante"
b = "irrelevante"

for tweet in train.Treinamento:
    tweet = clean(tweet)
    tweet = tweet.split()
    
    for i in range(len(tweet)):
        if tweet[i] in rel_relevante:
            prob_tweet_rel *= (abs_relevante[tweet[i]]+1)/(total_relevante+total2)
        else:
            prob_tweet_rel *= (0+1)/(total_relevante+total2)

        
        if tweet[i] in rel_irrelevante:
            prob_tweet_irr *= (abs_irrelevante[tweet[i]]+1)/(total_irrelevante+total2)
        else:
            prob_tweet_irr *= (0+1)/(total_irrelevante+total2)
            



            if (prob_tweet_rel*prob_rel) > (prob_tweet_irr*prob_irr):

                lista_rel.append("relevante")

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [55]:
total_classificado = len(train.Treinamento)
total_rel_classificado = len(lista_rel)
prob_rel_classificada = len(lista_rel)/len(train.Treinamento)
prob_irr_classificada = 1-((len(lista_rel)/len(train.Treinamento)))



print("o numero total de tweets no classificador é:", total_classificado)
print(55*"-")
print("o numero de tweets classificados como relevantes é: ", total_rel_classificado)
print(55*"-")
print("a probabilidade dos tweets serem relevantes é: ", prob_rel_classificada)
print(55*"-")
print("a probabilidade dos tweets serem irrelevantes é: ", prob_irr_classificada)

o numero total de tweets no classificador é: 500
-------------------------------------------------------
o numero de tweets classificados como relevantes é:  9
-------------------------------------------------------
a probabilidade dos tweets serem relevantes é:  0.018
-------------------------------------------------------
a probabilidade dos tweets serem irrelevantes é:  0.982


In [56]:
print("A probabilidade esperada de ser relevante é: ", prob_rel.round(2))
print(55*"-")
print("A probabilidade apos o classificador: ", prob_rel_classificada) 
print(55*"-")
print("Diferenca de: ", (prob_rel - prob_rel_classificada).round(2))
print(55*"-")
print("   ")
print("A probabilidade esperada de ser irrelevante é: ", prob_irr.round(2))
print(55*"-")
print("A probabilidade calculada pelo classificador: ", prob_irr_classificada) 
print(55*"-")
print("Diferenca de: ", (prob_irr_classificada - prob_irr).round(2))

A probabilidade esperada de ser relevante é:  0.58
-------------------------------------------------------
A probabilidade apos o classificador:  0.018
-------------------------------------------------------
Diferenca de:  0.57
-------------------------------------------------------
   
A probabilidade esperada de ser irrelevante é:  0.42
-------------------------------------------------------
A probabilidade calculada pelo classificador:  0.982
-------------------------------------------------------
Diferenca de:  0.57


___
### Concluindo

O classificador não funcionou como esperávamos, e após a análize manual dos tweets, acreditamos que a baixa perfoma do classificador se deve ao fato de que os usuários comentam pontos de vistas diferentes do produto, como jogabilidade, estética do produto, controle, jogos disponíveis, etc. E também, percebemos que os termos utilizados são muito diversos, por isso nossa concluimos também o motivo do classificador não gerar os resultados esperados.

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Analizar separado os diferentes aspectos do produto (games, imagem, controle, console, etc.)
* Criar categorias intermediárias de relevância;

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**